# Still a work in progress
Kevin Walchko
Created: 24 Oct 2016

# Denavit-Hartenberg (DH)

Development of the DH forward kinematics for a quadruped robot. We will use the DH method to develop the symbolic equations and use python to simplify them.

![](https://github.com/walchko/pyGeckoRobots/blob/master/quadruped/robotis/docs/spider.png?raw=true)



In [1]:
from __future__ import print_function
from __future__ import division
import numpy as np
from sympy import symbols, sin, cos, pi, simplify
from math import radians as d2r
from math import degrees as r2d
from math import atan2, sqrt, acos

In [2]:
t1, t2, t3 = symbols('t1 t2 t3')
Lc, Lf, Lt = symbols('Lc Lf Lt')

The following class follows the traditional [DH convention](https://en.wikipedia.org/wiki/Denavit%E2%80%93Hartenberg_parameters). Where:

| Parameter | Definition|
|-----------|-----------|
| $\alpha_i$   | twist |
| $d_i$        | dist |
| $\theta_i$   | angle |
| $a_i$        | length |

In [3]:
class DH(object):
    def __init__(self):
        pass

    def fk(self, params):
        t = np.eye(4)
        for p in params:
            t = t.dot(self.makeT(*p))
        return t

    def makeT(self, a, alpha, d, theta):
        # alpha = d2r(alpha)
        # theta = d2r(theta)
        return np.array([  # classic DH
            [cos(theta), -sin(theta) * cos(alpha),  sin(theta) * sin(alpha), cos(theta) * a],
            [sin(theta),  cos(theta) * cos(alpha), -cos(theta) * sin(alpha), sin(theta) * a],
            [         0,               sin(alpha),               cos(alpha),              d],
            [         0,                        0,                        0,              1]
        ])

In [4]:
def eval(f, inputs):
    h = []
    for i in range(0, 3):
        tmp = (f[i,3]).subs(inputs)
        h.append(tmp.evalf())
    return h

The parameters are:

| i |$a_i$| $\alpha_i$ | $d_i$ | $theta_i$ |
|---|-------|--------------|---------|-------------|
| 1 | coxa  | 90           | 0       | $\theta_1$ |
| 2 | femur | 0            | 0       | $\theta_2$ |
| 3 | tibia | 0            | 0       | $\theta_3$ |


In [5]:
# a, alpha, d, theta
params = [
    [Lc, pi/2, 0, t1],
    [Lf,    0, 0, t2],
    [Lt,    0, 0, t3]
]
dh = DH()
t = dh.fk(params)
t = eval(t,[])

In [6]:
for i in t:
    print(simplify(i))

1.0*(Lc + Lf*cos(t2) + Lt*cos(t2 + t3))*cos(t1)
1.0*(Lc + Lf*cos(t2) + Lt*cos(t2 + t3))*sin(t1)
1.0*Lf*sin(t2) + 1.0*Lt*sin(t2 + t3)


In [7]:
def fk(t1, t2, t3, Lc, Lf, Lt):
    t1 = d2r(t1)
    t2 = d2r(t2)
    t3 = d2r(t3)
    return [
        (Lc + Lf*cos(t2) + Lt*cos(t2 + t3))*cos(t1),
        (Lc + Lf*cos(t2) + Lt*cos(t2 + t3))*sin(t1),
        Lf*sin(t2) + 1.0*Lt*sin(t2 + t3)
    ]

# Inverse Kinematics

Often, we need to find the angles of the leg joints given a location (x,y,z) of the foot. To calculate this, remember the leg is composed of 3 revolute joints. The shoulder is in one plane and both the femur joint and tibia joint are in another plane.

**insert picture here**

## Useful Trigonometry

![](https://upload.wikimedia.org/wikipedia/commons/thumb/4/49/Triangle_with_notations_2.svg/200px-Triangle_with_notations_2.svg.png)

The **law of cosines** for calculating one side of a triangle when the angle
opposite and the other two sides are known. Can be used in conjunction with
the law of sines to find all sides and angles.

## Derivation

So let's start with the shoulder, look down from the top, we see the x-y plane. We can calculate the it's rotation $\theta_1$ by using x and y of the foot location.

\begin{equation}
    \theta_1 = atan2(y,x)
\end{equation}

Next, let's look at the plane which the next 2 joints lie in. Now we will calculate the $\theta_2$ and $\theta_3$ angles.

\begin{equation}
    m = \sqrt{x^2 + y^2} \\
	f = m - L_{coxa} \\
	\beta_1 = atan2(f,z) \\
	d = \sqrt{f^2 + z^2} \\
	\beta_2 = \arccos( \frac{L_{femur}^2+d^2-L_{tibia}^2}{2 d L_{femur}} ) \\
	\theta_2 = \beta_1 + \beta_2 \\
	\theta_3 = \arccos( \frac{L_{femur}^2+L_{tibia}^2-d^2}{2 L_{femur} L_{tibia}} )
\end{equation}

Now finally, you should notice the definition is different between forward and reverse kinematics. So let's fix that ...

\begin{equation}
    \theta_{2fk} = \theta_{2ik} - \pi \\
	\theta_{3fk} = \theta_{3ik} - \pi/2
\end{equation}

In [12]:
def ik(x, y, z, Lc, Lf, Lt):
    try:
        t1 = atan2(y, x)
        f = sqrt(x**2 + y**2) - Lc
        b1 = atan2(z, f)  # takes into accoutn quadrent, z is neg
        d = sqrt(f**2 + z**2)  # <---
        b2 = acos((Lf**2 + d**2 - Lt**2) / (2.0 * Lf * d))
        t2 = b1 + b2
        t3 = acos((Lf**2 + Lt**2 - d**2) / (2.0 * Lf * Lt))
        
        t3 -= pi  # fix to align fk and ik frames
#         t2 -= pi/2  # wtf ... i need this!!!!
            
        return [r2d(t1), r2d(t2),r2d(t3)]
    except Exception as e:
        print('ik error:', e)

pts = fk(0,0,0, 10, 40, 100)
print('pts: {} {} {}'.format(*pts))

angles = ik(pts[0], pts[1], pts[2], 10, 40, 100)
print('angles: {} {} {}'.format(*angles))

pts: 150 0 0
angles: 0.0 0.0 -7.01670929853e-15



-----------

<a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-sa/4.0/88x31.png" /></a><br />This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/">Creative Commons Attribution-ShareAlike 4.0 International License</a>.